# SoccerNet Per-Action Analysis

In [ ]:
# Read data
import pandas as pd

vlm_tsi_df = pd.read_csv("../soccernet+vlm-tsi+step=825+per_action.csv")
videollm_online_df = pd.read_csv(
    "../soccernet+videollm-online+threshold=default+per_action.csv"
)

In [ ]:
# print out actions
print(f"VLM-TSI: {vlm_tsi_df['action']}")
print(f"VideoLLM-Online: {videollm_online_df['action']}")
print(
    f"VLM-TSI - VideoLLM-Online: {set(vlm_tsi_df['action']) - set(videollm_online_df['action'])}"
)
print(
    f"VideoLLM-Online - VLM-TSI: {set(videollm_online_df['action']) - set(vlm_tsi_df['action'])}"
)

In [ ]:
# group actions
action_groups = {
    "Restarts": [
        "Kick-off",
        "Ball out of play",
        "Throw-in",
        "Corner",
        "Direct free-kick",
        "Indirect free-kick",
    ],
    "Attempts": ["Shots on target", "Shots off target", "Clearance"],
    "Infractions": ["Offside", "Foul"],
    "Discipline": ["Yellow card", "Red card", "Yellow->red card"],
    "Goal/Penalty": ["Goal", "Penalty"],
    "Substitution": ["Substitution"],
}

In [ ]:
# compute stats per action group
from pprint import pprint


def compute_stats(df):
    results = {}
    for group, actions in action_groups.items():
        subset = df[df["action"].isin(actions)]
        if subset.empty:
            continue
        results[group] = {
            "num_tasks": len(subset),
            "mean_semantic": float(subset["mean_acc_f1_adjusted"].mean()),
            "mean_timing": float(subset["mean_timing_f1_adjusted"].mean()),
            "TRACE": float(subset["final_score"].mean()),
        }
    return results


vlm_tsi_stats = compute_stats(vlm_tsi_df)
print("VLM-TSI")
pprint(vlm_tsi_stats)
videollm_online_stats = compute_stats(videollm_online_df)
print("VideoLLM-Online")
pprint(videollm_online_stats)

In [ ]:
# calculate VLM-TSI - VideoLLM-Online per task group
diff_per_task_group = {}
for task_group in vlm_tsi_stats:
    diff_per_task_group[task_group] = {}
    diff_per_task_group[task_group]["TRACE"] = (
        vlm_tsi_stats[task_group]["TRACE"] - videollm_online_stats[task_group]["TRACE"]
    )
    diff_per_task_group[task_group]["mean_semantic"] = (
        vlm_tsi_stats[task_group]["mean_semantic"]
        - videollm_online_stats[task_group]["mean_semantic"]
    )
    diff_per_task_group[task_group]["mean_timing"] = (
        vlm_tsi_stats[task_group]["mean_timing"]
        - videollm_online_stats[task_group]["mean_timing"]
    )
    diff_per_task_group[task_group]["num_tasks"] = vlm_tsi_stats[task_group][
        "num_tasks"
    ]
print("VLM-TSI - VideoLLM-Online")
pprint(diff_per_task_group)